# Day 10
## Part 1
Represent lights and buttons with sets and BFS. You could do this more efficiently with bitmasks as the lights and buttons are binary numbers.

In [17]:
from advent import read_input

def parse_data(s):
    data = []
    for line in s.strip().splitlines():
        fields = line.strip().split()
        lights = frozenset(
            i 
            for i, c in enumerate(fields[0][1:-1]) 
            if c == "#"
        )
        buttons = [
            frozenset(eval(field.replace(")", ",)")))
            for field in fields[1:-1]
        ]
        joltages = eval(fields[-1].replace("{", "[").replace("}", "]"))
        data.append((lights, buttons, joltages))
    return data

test_data = parse_data("""[.##.] (3) (1,3) (2) (2,3) (0,2) (0,1) {3,5,4,7}
[...#.] (0,2,3,4) (2,3) (0,4) (0,1,2) (1,2,3,4) {7,5,12,7,2}
[.###.#] (0,1,2,3,4) (0,3,4) (0,1,2,4,5) (1,2) {10,11,11,5,10,5}
""")

test_data

[(frozenset({1, 2}),
  [frozenset({3}),
   frozenset({1, 3}),
   frozenset({2}),
   frozenset({2, 3}),
   frozenset({0, 2}),
   frozenset({0, 1})],
  [3, 5, 4, 7]),
 (frozenset({3}),
  [frozenset({0, 2, 3, 4}),
   frozenset({2, 3}),
   frozenset({0, 4}),
   frozenset({0, 1, 2}),
   frozenset({1, 2, 3, 4})],
  [7, 5, 12, 7, 2]),
 (frozenset({1, 2, 3, 5}),
  [frozenset({0, 1, 2, 3, 4}),
   frozenset({0, 3, 4}),
   frozenset({0, 1, 2, 4, 5}),
   frozenset({1, 2})],
  [10, 11, 11, 5, 10, 5])]

In [19]:
from collections import deque

def min_presses(lights, buttons):
    q = deque([(frozenset(), 0)])
    seen = {frozenset()}
    while q:
        l, n = q.popleft()
        if l == lights:
            return n
        for b in buttons:
            new_ls = l ^ b
            if new_ls not in seen:
                q.append((new_ls, n + 1))
                seen.add(new_ls)

[min_presses(ls, bs) for ls, bs, _ in test_data]

[2, 3, 2]

In [20]:
def part_1(data):
    return sum(min_presses(ls, bs) for ls, bs, _ in data)

assert part_1(test_data) == 7

In [21]:
data = parse_data(read_input())

part_1(data)

484

## Part 2

Right, let's get something that looks like the above running before thinking if there's a better way.

In [36]:
from pyrsistent import pvector
import tqdm

def min_presses_j(buttons, joltages):
    target = pvector(joltages)
    q = deque([(pvector([0] * len(joltages)), 0)])
    seen = {pvector([0] * len(joltages))}
    while q:
        js, n = q.popleft()
        for b in buttons:
            new_js = js
            for i in b:
                new_js = new_js.mset(i, js[i] + 1)
            if new_js == target:
                return n + 1
            if new_js not in seen and not any(new_js[i] > target[i] for i in range(len(js))):
                q.append((new_js, n + 1))
                seen.add(new_js)

[min_presses_j(bs, js) for _, bs, js in test_data]

[10, 12, 11]

In [38]:
def part_2(data):
    result = 0
    for _, bs, js in tqdm.tqdm(data):
        result += min_presses_j(bs, js)
    return result

assert part_2(test_data) == 33

100%|█████████████████████████████████████████| 3/3 [00:00<00:00, 40.98it/s]


In [ ]:
part_2(data)

  0%|                                               | 0/186 [00:00<?, ?it/s]

This is going nowhere, I need to rethink.